22 et 23/06/2024, Nathan BONNEAU

#**Pricer de CDS et estimation de l'intensité de défaut à partir du prix des obligations**

# Importation des librairies

In [29]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

# Pricer de CDS

## Hypothèses :
- Les défauts peuvent survenir uniquement en milieu d'année
- Les paiements du CDS par l'acheteur sont faits une fois par an
- Le taux de recouvrement par défaut vaut 40% (valeur de l'obligation juste après le défaut)


À partir de l'intensité de défaut en entrée de la fonction, nous pouvons calculer la probabilité de défaut et la probabilité de survie pour chaque année.

Nous pourrons ensuite calculer la somme des cash flows actualisés payés par l'acheteur du CDS (dans *df_cds_survie*) et les actualiser (en posant s = spread que l'on recherche). En cas de défaut, l'acheteur du CDS aura payé uniquement la moitié du spread. Il faut donc calculer ces cash flows actualisés (dans *df_cds_defaut*).

On calculera ensuite les cash flows actualisés que pourrait générer un défaut que le vendeur devrait payer (dans *df_cds_defaut*) à l'acheteur du CDS.

Il restera à trouver s, le spread, tel que la somme des cash flows de l'acheteur actualisés soit égale à la somme des cash flows actualisés du vendeur.

In [32]:
def prixCDS(nbyears, intensityDefault, freeRisk, recouvrement =0.4):

  #CF lorsqu'il n'y a pas défaut de paiement
  df_cds_survie = pd.DataFrame()
  years = list(range(1, nbyears+1)) #dates (en nb de years)
  df_cds_survie['Maturite'] = years
  df_cds_survie['Proba de survie'] = np.exp(-intensityDefault*df_cds_survie['Maturite'])#proba de survie
  for index in df_cds_survie.index:
    if index == 0:
      df_cds_survie.loc[index,'Proba de défaut'] = 1 - df_cds_survie.loc[index,'Proba de survie']  #proba de défaut
    else:
     df_cds_survie.loc[index,'Proba de défaut'] =df_cds_survie.loc[index-1,'Proba de survie']-df_cds_survie.loc[index,'Proba de survie']
  df_cds_survie['Facteur actualisation'] = np.exp(-freeRisk*df_cds_survie['Maturite']) #facteur d'actualisation
  #paiement espéré actualisé de l'acheteur si il n'y a pas de défaut au cours de l'année (que l'on multiplie par le spread s)
  df_cds_survie['VA paiement acheteur'] = df_cds_survie['Proba de survie']*df_cds_survie['Facteur actualisation']

  #CF lorsqu'il y a défaut de paiement
  df_cds_defaut = pd.DataFrame() #dataframe des paiements espérés actualisés par le vendeur du CDS
  df_cds_defaut['Maturite'] = [x * 0.5 for x in range(1, nbyears * 2, 2)] #dates (en nb de years) en supposant que les défauts surviennent en milieu d'année
  df_cds_defaut['Proba de défaut'] =  df_cds_survie['Proba de défaut']
  df_cds_defaut['Recouvrement'] = recouvrement #valeur de l'oblig juste après le défaut
  df_cds_defaut['Facteur actualisation'] = np.exp(-freeRisk*df_cds_defaut['Maturite'])
  df_cds_defaut['VA paiement vendeur'] = df_cds_defaut['Proba de défaut']*df_cds_defaut['Facteur actualisation']*(1-recouvrement)
  df_cds_defaut['VA paiement acheteur (cas defaut)'] = df_cds_defaut['Proba de défaut']*df_cds_defaut['Facteur actualisation']*0.5

  #affichage des deux dataframes
  #print(df_cds_survie)
  #print(df_cds_defaut)

  #calcul du spread
  paiement_acheteur = df_cds_survie['VA paiement acheteur'].sum() + df_cds_defaut['VA paiement acheteur (cas defaut)'].sum()   #valeur totale actuelle du paiement de la part de l'acheteur du CDS
  paiement_vendeur = df_cds_defaut['VA paiement vendeur'].sum()   #valeur totale actuelle du paiement de la part du vendeur de CDS
  #il faut maintenant égaliser ces deux montants
  spread =  paiement_vendeur/ paiement_acheteur
  print("Le spread du CDS associé à une maturité dans", nbyears, "ans, une intensité de défaut de", intensityDefault, "%, pour un taux sans risque de ", freeRisk, "% est de", round(spread*10000), "bp.")
  return spread



nbyears = int(input("Veuillez entrer la maturité associé à votre CDS : "))
intensityDefault = float(input("Veuillez entrer l'intensité de défaut : "))
riskFree = float(input("Veuillez entrer le taux sans risque : "))
recouvrement = float(input("Veuillez entrer le recouvrement : "))
spread = prixCDS(nbyears, intensityDefault, riskFree, recouvrement)


Veuillez entrer la maturité associé à votre CDS : 5
Veuillez entrer l'intensité de défaut : 0.02
Veuillez entrer le taux sans risque : 0.05
Veuillez entrer le recouvrement : 0.4
Le spread du CDS associé à une maturité dans 5 ans, une intensité de défaut de 0.02 %, pour un taux sans risque de  0.05 % est de 123 bp.


# Estimation de l'intensité de défaut à partir des prix d'obligations

Inspiration des calculs de l'exemple 24.2 du *Hull*, code Python permettant d'appliquer ces calculs sur un champ plus large d'obligations et d'obtenir le résultat facilement.

## Hypothèses :
- Chaque obligation a un nominal de 100€
- Le taux de coupon en paramètre est annuel
- Les coupons sont payés semestriellement
- Les dates de défauts ne peuvent survenir que dans 3 mois ou 9 mois


L'objectif est à partir des taux actuariels des obligations (pour différentes maturités), du taux sans risque, de la maturité souhaitée, du recouvrement et du taux de coupon annuel pouvoir retrouver et estimer l'intensité de défaut pour chaque maturité et pouvoir s'en servir pour pricer un CDS.

## Fonction objective

*objective_function* est une fonction que nous allons utiliser avec de multiples itérations pour faire **égaliser le coût de défaut** (différence entre la valeur de l'obligation actualisée et la valeur de l'obligation au taux sans risque) avec la somme des **pertes espérées** (la probabilité de défaut multipliée par la valeur actualisée de la perte en cas de défaut).

In [35]:
def objective_function(x,df_defaut,i, cout_defaut):
  if(i==1): #maturité d'1 an
    df_defaut.loc[0,'Probabilité de survie'] = np.exp(-x * 0.5)
    df_defaut.loc[1,'Probabilité de survie'] = df_defaut.loc[0,'Probabilité de survie']*np.exp(-x * 0.5)
    df_defaut.loc[0,'Probabilité de defaut'] = 1 - df_defaut.loc[0,'Probabilité de survie']
    df_defaut.loc[1,'Probabilité de defaut'] = df_defaut.loc[0,'Probabilité de survie']-df_defaut.loc[1,'Probabilité de survie']
  else : #cas des maturités plus d'1 an
    df_defaut.loc[2 * (i - 1),'Probabilité de survie'] = df_defaut.loc[2 * (i - 1)-1,'Probabilité de survie']*np.exp(-x * 0.5)
    df_defaut.loc[2 * (i - 1)+1,'Probabilité de survie'] = df_defaut.loc[2 * (i - 1),'Probabilité de survie']*np.exp(-x * 0.5)
    df_defaut.loc[2 * (i - 1),'Probabilité de defaut'] = df_defaut.loc[2 * (i - 1)-1,'Probabilité de survie']-df_defaut.loc[2 * (i - 1),'Probabilité de survie']
    df_defaut.loc[2 * (i - 1) + 1,'Probabilité de defaut'] = df_defaut.loc[2 * (i - 1) ,'Probabilité de survie']-df_defaut.loc[2 * (i - 1) + 1,'Probabilité de survie']
  df_defaut['Perte espérée'] = df_defaut['VA de la perte'] * (df_defaut['Probabilité de defaut'])
  sum_loss = df_defaut['Perte espérée'].sum() #somme des pertes espérées
  return sum_loss - cout_defaut #il faut que cette différence tends vers 0


*compute_with_optimal_parameters* est une fonction similaire à objective_function mais qui **retourne les probabilités de survie et de défaut optimales** une fois que nous avons trouvé l'intensité de défaut de la maturité à la i-ème année. Elle permet de renvoyer les probabilités de survie et de défaut optimales trouvées et de pouvoir s'en servir pour les maturités prochaines.

In [36]:
def compute_with_optimal_parameters(x,df_defaut, i):
    if(i==1):#maturité d'1 an
      df_defaut.loc[0,'Probabilité de survie'] = np.exp(-x * 0.5)
      df_defaut.loc[1,'Probabilité de survie'] = df_defaut.loc[0,'Probabilité de survie']*np.exp(-x * 0.5)
      df_defaut.loc[0,'Probabilité de defaut'] = 1 - df_defaut.loc[0,'Probabilité de survie']
      df_defaut.loc[1,'Probabilité de defaut'] = df_defaut.loc[0,'Probabilité de survie']-df_defaut.loc[1,'Probabilité de survie']
    else : #cas des maturités plus d'1 an
      df_defaut.loc[2 * (i - 1),'Probabilité de survie'] = df_defaut.loc[2 * (i - 1)-1,'Probabilité de survie']*np.exp(-x * 0.5)
      df_defaut.loc[2 * (i - 1)+1,'Probabilité de survie'] = df_defaut.loc[2 * (i - 1),'Probabilité de survie']*np.exp(-x * 0.5)
      df_defaut.loc[2 * (i - 1),'Probabilité de defaut'] = df_defaut.loc[2 * (i - 1)-1,'Probabilité de survie']-df_defaut.loc[2 * (i - 1),'Probabilité de survie']
      df_defaut.loc[2 * (i - 1) + 1,'Probabilité de defaut'] = df_defaut.loc[2 * (i - 1) ,'Probabilité de survie']-df_defaut.loc[2 * (i - 1) + 1,'Probabilité de survie']
    print(df_defaut)
    df_defaut['Perte espérée'] = df_defaut['VA de la perte'] * (df_defaut['Probabilité de defaut'])
    sum_loss = df_defaut['Perte espérée'].sum()
    tab_survie  = df_defaut['Probabilité de survie']
    tab_defaut  = df_defaut['Probabilité de defaut']
    return tab_survie, tab_defaut

## Fonction qui estime l'intensité de proba de défaut avec le prix des obligations

*estimate_defaut_bondsprice* est la fonction qui renvoie une liste des taux d'intensité jusqu'à la maturité souhaitée en essayant d'égaliser le coût de défaut et la somme des pertes espérées pour retrouver ce taux d'intensité à partir des taux actuariels.

In [41]:
def estimate_defaut_bondsprice(nbyears, freeRisk, recouvrement, taux_coupon):
  df_actuariel = pd.DataFrame()

  #demande à l'utilisateur les taux actuariels pour chaque maturité de l'obligation
  for i in range(nbyears):
        taux = float(input(f"Entrez le taux actuariel de l'obligation à {i+1} an(s): "))
        df_actuariel.loc[i,'Maturité'] = f"{i+1} an(s)"
        df_actuariel.loc[i,'Taux actuariel'] = taux
  clear_output()

  #small résumé des parameters
  print(f"Voici les inputs que nous allons utiliser pour déterminer le taux d'intensité de défaut pour {nbyears} an(s)")
  print("Taux sans risque :", freeRisk )
  print("Coupon annuel :", taux_coupon*100)
  print("Recouvrement :", recouvrement)
  print("Taux actuariel de lobligation pour différentes maturités:")
  print(df_actuariel)

  #remplissage du dataframe df qui permet de calculer le cout de défaut à partir de la valeur de l'obligation actualisée avec les taux actuariels et la valeur de  l'obligation actualisée avec le freeRisk
  index = 0
  cout_defaut_list = []
  tab_survie = []
  for i in range(1,nbyears+1):
    df = pd.DataFrame()
    dates = [j / 2 for j in range(1, ((i)*2)+1)]
    df['Date'] = dates
    j =0
    a = 0
    while j <len(dates)-1:
      df.loc[j,'Taux actuariel'] = df_actuariel.loc[a,'Taux actuariel'] #mettre une colonne avec le taux actuariel correspondant à la date
      df.loc[j+1,'Taux actuariel'] = df_actuariel.loc[a,'Taux actuariel']
      j = j+2
      a = a+1
    cf = [(taux_coupon / 2) * 100 for j in range(1, ((i)*2))]
    cf.append(100 + ((taux_coupon / 2)*100))
    df['Cash Flow'] = cf
    df['Valeur de lobligation'] = df['Cash Flow'] * np.exp(-df_actuariel.loc[index,'Taux actuariel']*df['Date'])
    df['Valeur hors défaut'] = df['Cash Flow'] * np.exp(-freeRisk*df['Date'])
    df.loc['Total','Valeur de lobligation' ] = df['Valeur de lobligation'].sum()
    df.loc['Total','Valeur hors défaut' ] = df['Valeur hors défaut'].sum()
    df.loc['Cout de défaut', 'Total'] = df.loc['Total','Valeur hors défaut'] - df.loc['Total','Valeur de lobligation']
    cout_defaut = df.loc['Cout de défaut', 'Total']
    index +=1
    cout_defaut_list.append(cout_defaut)

  results = []
  for i in range(1,nbyears+1):
    print("i==================",i)
    df_defaut = pd.DataFrame()
    df_defaut['Date'] =  np.arange(0.25, i, 0.5) #générer les dates potentiels de défaut de paiements
    if(i!=1):
        index = 0
        while index < len(tab_survie):
            df_defaut.loc[index, 'Probabilité de survie'] = tab_survie[index]
            df_defaut.loc[index, 'Probabilité de defaut'] = tab_defaut[index]
            index += 1

    n =len(df_defaut['Date'])
    for t in range(n - 1, -1, -1):
      if(t==int(len(df_defaut['Date'])-1)):
        df_defaut.loc[t,'Valeur hors defaut'] =  ((taux_coupon/2)*100 +100)*np.exp(-freeRisk*0.25) #valeur si pas de défaut
      else :
        df_defaut.loc[t,'Valeur hors defaut'] =  (taux_coupon/2)*100 *np.exp(-freeRisk*0.25)+df_defaut.loc[t+1,'Valeur hors defaut']*np.exp(-freeRisk*(df_defaut.loc[t+1,'Date']-df_defaut.loc[t,'Date']))
    df_defaut['recouvrement'] = recouvrement*100
    df_defaut['perte si defaut'] = df_defaut['Valeur hors defaut'] - df_defaut['recouvrement'] #montant de la perte après défaut
    df_defaut['VA de la perte'] = df_defaut['perte si defaut']*np.exp(-freeRisk*df_defaut['Date']) #actualisation de la perte

    #au lieu d'utiliser un optimiseur python (qui sont moins forts que celui d'excel), j'ai préféré faire une boucle avec un pas de 1bp
    x_values = np.arange(0.01, 1.01, 0.0001)
    min_value = float('inf')
    optimal_x = None
    for x in x_values:
        current_value = objective_function(x, df_defaut.copy(),i,cout_defaut_list[i-1])
        if abs(current_value) < abs(min_value):
            min_value = current_value
            optimal_x = x

    #on stocke les taux d'intensité de défaut dans la tableau results
    results.append(optimal_x)
    tab_survie, tab_defaut = compute_with_optimal_parameters(optimal_x, df_defaut,i)
    print(f"Year {i}: Optimal x = {optimal_x}")
  dates = [f"{i+1} an{'s' if i > 0 else ''}" for i in range(len(results))]
  df_taux_intensite_defaut = pd.DataFrame({'Date': dates,'Result': results})
  return df_taux_intensite_defaut


nbyears = int(input("Veuillez entrer la maturité dont vous souhaitez connaître l'intensité de défaut  : "))
Freerisk =  float(input("Veuillez entrer le taux sans risque : "))
recouvrement = float(input("Veuillez entrer le recouvrement : "))
taux_coupon= float(input("Veuillez entrer le taux de coupon annuel sur votre bond: "))

df_taux_intensite_defaut = estimate_defaut_bondsprice(nbyears,Freerisk ,recouvrement, taux_coupon)
df_taux_intensite_defaut



Voici les inputs que nous allons utiliser pour déterminer le taux d'intensité de défaut pour 3 an(s)
Taux sans risque : 0.05
Coupon annuel : 8.0
Recouvrement : 0.4
Taux actuariel de lobligation pour différentes maturités:
  Maturité  Taux actuariel
0  1 an(s)          0.0650
1  2 an(s)          0.0680
2  3 an(s)          0.0695
i================== 1
   Date  Valeur hors defaut  recouvrement  perte si defaut  VA de la perte  \
0  0.25          104.122531          40.0        64.122531       63.325988   
1  0.75          102.708091          40.0        62.708091       60.400083   

   Probabilité de survie  Probabilité de defaut  
0               0.987775               0.012225  
1               0.975700               0.012075  
Year 1: Optimal x = 0.024599999999999914
i================== 2
   Date  Probabilité de survie  Probabilité de defaut  Valeur hors defaut  \
0  0.25               0.987775               0.012225          106.847504   
1  0.75               0.975700               0

,Date,Result
0,1 an,0.0246
1,2 ans,0.0347
2,3 ans,0.0374
